
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Part 1</h1>

<h1 style="font-family:verdana;font-size:150%;text-align:Center;color:#993333"> Lab 1: Introduction to text classification  </h1>

<h1 style="font-family:verdana;font-size:150%;text-align:left;color:blue">Section 1: Text Classification with Naive Bayes Classifier </h1>

##### In this part you'll implement Naive Bayes classifier to classify the text. You need to build a model that predicts the langauge of the text given the words of the text

In [10]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [11]:
def load_data(filename):
    '''
    Parameters:
    filename (string): path to file to be read
    
    Return: 
    List of tuples (explained in first question)
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    for line in fin:
        tokens = line.split()
        data.append((tokens[0], tokens[1:]))
    return data

In [12]:
data = load_data("train1.txt")
data[0]
# Tuple 

('__label__deu',
 ['Ich', 'würde', 'alles', 'tun,', 'um', 'dich', 'zu', 'beschützen.'])

In [13]:
def count_words(data):
    '''
    Parameters:
    
    data is  list of [(label, words), (label, worlds), ......]
    list of tuples in the shape (string, [list of strings]) )
    
    Returns: 
    
    This function should return a dictionary containing the following:
    { 
    # label_counts (python dictionary): 
         {label:  no. of times the label appeared },
    # word_counts  (dictionary of dictionaries): 
         {label: {word: no. of times this word appeared with this label }},
    # label_total (int): 
        total number of labels. (size of train data),
    # word_total  (python dictionary) total number of words (from the entire corupus) of the particular label:
          {label: no.of words}
          
          }
    
    '''
    label_total = 0
    word_total = defaultdict(lambda: 0)
    label_counts = defaultdict(lambda: 0)
    word_counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for example in data:
        label, sentence = example
        
        ## FILL CODE
        label_counts[label] += 1.0
       
        
        for word in sentence:
            word_counts[label][word] += 1.0
            word_total[label] += 1.0
              
    label_total = len(label_counts.keys())
      

    return {'label_counts': label_counts, 
            'word_counts': word_counts, 
            'label_total': label_total, 
            'word_total': word_total}

In [14]:
result = count_words(data)

label_counts = result['label_counts']
word_counts = result['word_counts']
label_total = result['label_total']
word_total = result['word_total']

#label_counts.keys()
label_counts


defaultdict(<function __main__.count_words.<locals>.<lambda>>,
            {'__label__deu': 828.0,
             '__label__hun': 432.0,
             '__label__rus': 1271.0,
             '__label__ita': 1327.0,
             '__label__eng': 2137.0,
             '__label__spa': 564.0,
             '__label__tur': 1193.0,
             '__label__epo': 1020.0,
             '__label__por': 578.0,
             '__label__fra': 650.0})

In [15]:
import operator
def predict(sentence, mu, label_counts, word_counts, label_total, word_total):
    '''
     Parameters: 
        sentence (string): sentence to be classified
        mu (positive real number): Laplace Smoothing hyperparameter
        ** The other parameters introduced in the count_words function
    
    Returns:
    best_label (string): the label that has the highest score. 
    
    Implement the function to predict the best label for the given sentence using Naive Bayes algorithm 
    
    '''
    best_label = None
    best_score = float('-inf')
    
    scores_list = {}
    for label in word_counts.keys():
        score = 0.0
    
                                         ## FILL CODE 
    
        # probability of class
        p_c = label_counts[label]/sum(label_counts.values())
        
        # sum of all words in given class
        w_c = sum(word_counts[label].values())
        
         # computing likelihood, p(w|c)
        product = 1
        for word in sentence:  
            product *=  (word_counts[label][word]+mu)/(w_c+(mu*len(word_counts['__label__deu'].keys())))
            
        # p(w|c) * p(class)
        scores_list[label] = product * p_c
            
        
    # getting the maximum probability
    sorted_x = sorted(scores_list.items(), key=lambda x: x[1], reverse=True)
    
    best_label = sorted_x[0][0]
    best_score = sorted_x[0][1]
    

    return best_label


In [16]:
count = 0
mu = 1
for example in data:
        label, sentence = example
        print(' '.join(sentence),predict(sentence, mu, label_counts, word_counts, label_total, word_total))
        count += 1
        if count == 5:
            break

Ich würde alles tun, um dich zu beschützen. __label__deu
Tom ist an Kunst völlig uninteressiert. __label__deu
Végeztem Tomival. __label__hun
„Wird das in der Werkstatt gemacht?“ – „Nein, das muss an Ort und Stelle erledigt werden.“ __label__deu
У меня есть яблоко. __label__rus


In [17]:
def compute_accuracy(valid_data, mu, counts):
    '''
    Parameters:
    valid_data (list of tuples): returned value of load_data function 
    mu (positive real): Laplace smoothing hyper-parameter
    counts (dictionary of dictionaries): return value of count_words_function
    
    Returns: 
    accuracy (float): the accuracy of the Naive Bayes classifier
    '''
    accuracy = 0.0
    for label, sentence in valid_data:
        predicted = predict(sentence, 1, label_counts, word_counts, label_total, word_total)         
        if predicted == label:
        #(predicted == label)*1
            accuracy += 1.0

    return accuracy/len(valid_data)


In [18]:
print("")
print("** Naive Bayes **")
print("")

mu = 1.0
train_data = load_data("train1.txt")
valid_data = load_data("valid1.txt")
counts = count_words(data)

print("Validation accuracy: %.3f" % compute_accuracy(valid_data, mu, counts))
print("")



** Naive Bayes **

Validation accuracy: 0.953



<h1 style="font-family:verdana;font-size:150%;text-align:left;color:black">***************************************************************</h1>


<h1 style="font-family:verdana;font-size:150%;text-align:left;color:blue">Section 2: Softmax Classification of Text  </h1>

##### In this part you'll implement a Softmax Classifier to classify the text (think of it as a 1 layer feedforward neural network). You need to build a model that predicts the langauge of the text given the words of the text

In [19]:
def build_dict(filename, threshold=1):
    '''
    Parameters:
    filename (string): path to the data file
    
    Returns:
    word_dic: dictionary maps words to number of times it appeard in the corpus
            dic {word: no of times word appears }
    label_dic: dictionary maps labels to integers
        dic {label: label_id}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

In [20]:
def load_data(filename, word_dict, label_dict):
    '''
    ## This function converts the text to a list of tuples of 
    [(label_id, word_representation),...]
    
    Parameters:
    filename (string): path to the file which contains the data
    word_dict (python dictionary): returned by build_dict() function above.
    label_dict (python dictionary): reutrned by build_dict() function above() 
    
    Returns:
    data (list of tuples): 
    The representation of the data in the form 
    [(y_0, x_0, .. (y_i, x_i), ... (y_n, x_n))]
    where y is the value of the class 
    x is the representation of the sentence as a word count vector 
    
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

In [22]:
word_dict, label_dict = build_dict("train1.txt", threshold=1)
d = load_data("train1.txt", word_dict, label_dict)

In [23]:
d[:5]

[(0, array([1., 1., 1., ..., 0., 0., 0.])),
 (0, array([0., 0., 0., ..., 0., 0., 0.])),
 (1, array([0., 0., 0., ..., 0., 0., 0.])),
 (0, array([0., 0., 0., ..., 0., 0., 0.])),
 (2, array([0., 0., 0., ..., 0., 0., 0.]))]

In [24]:
def softmax(x):
    '''
    This function should apply softmax to vector x
    
    Parameter:
    x (numpy array)
    Returns: 
    softmax(x) (numpy array)
    
    '''
    ## FILL CODE
    m = max(x)
    

    return np.exp(x - m)/np.sum(np.exp(x-m))

## (Hint) Derivatives:

<h1 style="font-family:verdana;font-size:150%;text-align:center;background-color:#f2f2f2;color:#993333; border:2px; border-style:solid; border-color:gray; padding: 1em"> 
   Let $x_i$ be the input vector $W$ is the weight vector $m=$no. of labels, $n=$vocab size
    $$ {\bf S} = W × x_i $$  $x_{i} \in R^{nx1} W \in R^{m×n}$
    $$  $$
    $${\bf O} = softmax(s) $$
    $${\bf L} = -log(O[y_{i}]) $$
    $$  $$
    $$\frac{\partial L}{\partial W} = \frac{\partial L}{\partial S} . \frac{\partial S}{\partial W} $$
    $$  $$
    $ \nabla L_{W} = (O-y_{true})$  x   $x_{i}^{T} $  
    $$ (O-y_{true}) \in R^{mx1}, x_{i}  \in R^{nx1}$$

</h1>

In [110]:
def sgd(w, data, niter):
    '''
    This function should perform the Stochastic Gradient Descent algorithm 
    
    Parameter:
    w (numpy array): weight vector
    data (list of tuples): [...(y_i, x_i)...] from above
    niter (int): number of iterations
    
    Retunrs:
    w (numpy array): weight vector after training
    '''
    nlabels, dim = w.shape
    lr = 0.01
    for iter in range(niter):
        ## FILL CODE
        for d in data:
            y, x = d
            y = np.eye(nlabels)[y]
            s = np.dot(w, x)
            o = softmax(s)
            l = -np.log(o)
            dL = (o - y).reshape(-1,1) * x.reshape(-1,1).T
         
            w = w - lr * dL
            
        
        
    return w

In [111]:
def predict(w, x):
    '''
    This function should compute and return the prediction. 
    Parameters:
    w (numpy array): trained weight vector
    x (numpy array): word count vector
    
    Returns: 
    prediction (int): index of the correct prediction (y_i)
    '''
    ## FILL CODE
    #w = sgd(w, data, niter)
    prediction = np.argmax(np.dot(w, x))
    return prediction

In [112]:
def compute_accuracy(w, valid_data):
    '''
    This function should compute the accuracy of the classifier 
    Parameters:
    w (numpy array): trained weight vector
    valid_data (list of tuples): loaded validation data using load_data() function 
    
    Returns: 
    accuracy (float): accuracy of the classifier 
    '''
    ## FILL CODE
    
    accuracy = 0.0
    for y, x in valid_data:
        predicted = predict(w, x)       
        if predicted == y:
       
            accuracy += 1.0

    return accuracy/len(valid_data)

   

In [114]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 50)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 0.922

